# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [3]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
maxT = 3*oneDay
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

# Run this section to compute your database
    You will be prompt to provide a database

In [ ]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis.Util import getAllEvents
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventWaterPoint, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventStop,
                  BuildEventWaterPoint,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    # CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    # update missing fields
    try:
        connection = sqlite3.connect( file )    
        c = connection.cursor()            
        query = "ALTER TABLE EVENT ADD METADATA TEXT";
        c.execute( query )    
        connection.commit()
        c.close()
        connection.close()
    except:
        print( "METADATA field already exists" , file )
        
    #build index
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    '''
    # build sensor data    
    animalPool = AnimalPool( )
    animalPool.loadAnimals( connection )
    animalPool.buildSensorData(file)
    '''
    
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
     

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 17.078222848
A window is popping (maybe hidden) asking for files to process...
Processing file C:/Users/Fab/Desktop/Experiment 8466.sqlite
C:/Users/Fab/Desktop/Experiment 8466.sqlite
Automatic settings.
Auto max set to 245722 frames
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAMENUMBER` ASC);
index detectionIndex already exists
CREATE INDEX `detetIdIndex` ON `DETECTION` (`ID` ASC);
index detetIdIndex already exists
CREATE INDEX `detframenumberIndex` ON `DETECTION` (`FRAMENUMBER` ASC);
index detframenumberIndex already exists
CREATE INDEX `eventEndFrameIndex` ON `EVENT` (`ENDFRAME` ASC);
index eventEndFrameIndex already exists
CREATE INDEX `eventIndex` ON `EVENT` (`ID` ASC,`STARTFRAME` ASC,`ENDFRAME` ASC);
index eventIndex already exists
CREATE INDEX `eventStartFrameIndex` ON `EVENT` (`STARTFRAME` ASC);
index eventSt

Number of event deleted: 0 
Deleting event Stop isolated (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Stop isolated"
Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventStop' from '..\\lmtanalysis\\BuildEventStop.py'>  ]  0.1399984359741211  seconds
Deleting event Water Zone (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Zone"
Number of event deleted: 0 
Deleting event Water Stop (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Stop"
Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventWaterPoint' from '..\\lmtanalysis\\BuildEventWaterPoint.py'>  ]  0.12799906730651855  seconds
Deleting event Approach contact (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Approach contact"
Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventApproachContact' from '..\

Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:1 idB:4 idC:None idD:None loadEvent: False
Number of event:  1347
Mean length of event:  4.534521158129176
first event frame:  1553
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 1347 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:1 idC:None idD:None loadEvent: False
Number of event:  984
Mean length of event:  4.7266260162601625
first event frame:  839
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 984 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  917
Mean length of event:  4.708833151581243
first event frame:  321
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 917 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:4 idC:None idD:None loa

Number of event:  1022
Mean length of event:  6.169275929549902
first event frame:  3636
Keep previous entry.
Saving timeLine: Side by side Contact ( 1022 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  1030
Mean length of event:  6.5
first event frame:  109
Keep previous entry.
Saving timeLine: Side by side Contact ( 1030 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  1100
Mean length of event:  7.345454545454546
first event frame:  2370
Keep previous entry.
Saving timeLine: Side by side Contact ( 1100 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  1160
Mean length of event:  7.323275862068965
first event frame:  1431
Keep 

Oral-genital Contact  Id( 2 , 4 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1529  records loaded in  0.298994779586792 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 1 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1403  records loaded in  0.2839930057525635 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 2 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 985  records loaded in  0.2930011749267578 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 4 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1909  records loaded in  0.29698634147644043 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 1 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1714  records loaded in  0.2919950485229492 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 2 , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1484  records loaded in  0.29001617431640625 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 3 , None , None ) Min/maxFrame: ( 

Stop  Id( 3 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 6171  records loaded in  0.40601325035095215 S )
Detection  Id( 3 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 476  records loaded in  0.4729800224304199 S )
Contact  Id( 3 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1376  records loaded in  0.3710181713104248 S )
Caching eventTimeLine
Stop  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 6200  records loaded in  0.4609856605529785 S )
Detection  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 369  records loaded in  0.44301414489746094 S )
Contact  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 245722 ) Loaded ( 1121  records loaded in  0.37999582290649414 S )
Caching eventTimeLine
Event Move created. eventNameWithId = Move idA:1 idB:None idC:None idD:None loadEvent: False
Number of event:  6800
Mean length of event:  6.476764705882353
first event frame:  33
Keep previous entry.
Saving timeL

Number of event:  679
Mean length of event:  1.796759941089838
first event frame:  669
Keep previous entry.
Saving timeLine: FollowZone Isolated ( 679 events )
A follow B
FollowZone Isolated
Event FollowZone Isolated created. eventNameWithId = FollowZone Isolated idA:1 idB:3 idC:None idD:None loadEvent: False
FollowZone Isolated idA:1 idB:3 idC:None idD:None
Number of event:  681
Mean length of event:  2.1116005873715125
first event frame:  53
Keep previous entry.
Saving timeLine: FollowZone Isolated ( 681 events )
A follow B
FollowZone Isolated
Event FollowZone Isolated created. eventNameWithId = FollowZone Isolated idA:2 idB:3 idC:None idD:None loadEvent: False
FollowZone Isolated idA:2 idB:3 idC:None idD:None
Number of event:  639
Mean length of event:  2.0469483568075115
first event frame:  1452
Keep previous entry.
Saving timeLine: FollowZone Isolated ( 639 events )
A follow B
FollowZone Isolated
Event FollowZone Isolated created. eventNameWithId = FollowZone Isolated idA:4 idB:3 